# GoogleEarthEngine Python Image Download
In this notebook we will download geodata from GEE.

For this we will need the following libraries:

In [ ]:
import ee
import geemap
import geopandas as gpd
import os

#os.chdir("...")

Once you have created your GEE-Account you can start the authentication process with the following lines of code. Then a browser window will open automatically where you can copy a token. Please make sure that you check all the terms and conditions boxes, otherwise the api-code might not work. You also have to register your cloud project for non commercial use.

In [ ]:
#Google Earth Authenticate Process - API-Code is needed
ee.Authenticate()
ee.Initialize()

Now we start with the download of a ESA-WorldCover image. This data is a global landcover classification in a 10x10m resolution. First we need a study area again. We will use the city bremen. The city file is divided in all its different districts or neighborhoods. To get a single merged area we will use the dissolve function.

In [ ]:
inpath = "../data_example/Example_Bremen_Neighborhoods.gpkg"
city = gpd.read_file(inpath)
dissolve = city.dissolve()

Now we have to convert the geodataframe in a Earth Engine readable format. Then we can pick a Image as a Image Collection.

In [ ]:
ee_readable = geemap.geopandas_to_ee(dissolve)
esa = ee.ImageCollection("ESA/WorldCover/v100").filterBounds(ee_readable.geometry())

Now we just have to declare an output path or a name for our file and export the image via ee. The important part here is to set the right geometric scale and a fitting coordinate reference system!

In [ ]:
outpath = "Worldcover_Bremen.tif"
geemap.ee_export_image(esa.first(), 
                        filename= outpath,
                        scale = 10, region = ee_readable.geometry(), file_per_band=False,
                        crs = 'EPSG:3035')

Great. Now let's build a function out of it und use it in other scripts too! There are many other datasets to download to play with or to analyze. Just explore their website with good documented tutorials on https://earthengine.google.com/

In [ ]:
def download_WorldCover(inpath,outpath):
    '''
    Downloads World Cover Data from Google Earth Engine API. Input file must be a boundary file as GPKG or SHP.

    Parameters
    ----------
    inpath : STRING
        File path including filename for area of interest. Must be GPKG or SHP.
    outpath : STRING
        File path including filename for WorldCover out file. Don't forget .tif extension!

    Returns
    -------
    None.

    '''
    
    city = gpd.read_file(inpath)
    dissolve = city.dissolve()
    ee_readable = geemap.geopandas_to_ee(dissolve)
    esa = ee.ImageCollection("ESA/WorldCover/v100").filterBounds(ee_readable.geometry())
    geemap.ee_export_image(esa.first(), 
                            filename= outpath,
                            scale = 10, region = ee_readable.geometry(), file_per_band=False,
                            crs = 'EPSG:3035')
    
    
if __name__ == '__main__':
    download_WorldCover("CityDistricts/Districts_FrankfurtAmMain.gpkg",
                        "WorldCover/WorldCover_FrankfurtAmMain.tif")